In [ ]:
# This is the chatbot

In [ ]:
from dotenv import load_dotenv
import websockets
import json
import asyncio
import requests

load_dotenv()

import os
oopenai_api_key = os.environ.get("OPENAI_API_KEY")
BOT_TOKEN = "5a3556bb2de44a73ab2e5643cb633a6c"
THREAD_ID = "default"

In [ ]:
def get_gpt3_response(prompt, stop=None):
    headers = {"Authorization": f"Bearer {oopenai_api_key}", "Content-Type": "application/json"}
    data = {"prompt": prompt, "max_tokens": 500, "temperature": 0, "model": "text-davinci-002"}
    if stop:
        data["stop"] = stop
    response = requests.post("https://api.openai.com/v1/completions", headers=headers, json=data)
    return response.json()["choices"][0]["text"]

talking_prompt = "This is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly. Sometimes the AI will say a crude joke in the form of a pun."

async def chatbot():
    uri = f"ws://localhost:8000/ws/chat?thread_id={THREAD_ID}"
    messages = []
    async with websockets.connect(uri, extra_headers={"Cookie": f"Authorization=Bearer {BOT_TOKEN}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                print("log:", data['message'])
                if data.get("replay", False):
                    messages.append(data)
                    continue
                if data["message"] == "break":
                    break
                if data["sender"] == "datAI":
                    # personal messages
                    continue
                if data.get("meta", False):
                    continue
                messages.append(data)
            except asyncio.TimeoutError:
                # not sure why I'm doing this, it felt important to have an "infinite" loop...
                continue
            # Valid message to parse and likely respond to?
            prompt_to_send = talking_prompt + "\n" + "\n".join([m["sender"]+":"+m["message"] for m in messages])+"\n"+"datAI:"
            response = get_gpt3_response(prompt_to_send)
            await ws.send(json.dumps({"message": response}))


In [ ]:
await chatbot()